# Download a random pro's CFG and place it in CSGO folder
## Set CSGO Path

In [5]:
# Change to match your CSGO install directory
PATH = "H:\\SteamLibrary\\steamapps\\common\\Counter-Strike Global Offensive"

## Download/Create Dataframe

In [6]:
from download_table import get_table
dataframe = get_table()

# Filter out rows where "Team" column isn't one of:
# "FaZe Clan", "NaVi", "Cloud9", "ENCE", "Astralis", "Vitality", "MR", "G2", "NiP", "MOUZ", "FURIA", "BIG", "Heroic", "Liquid" or "Spirit"
team_list = ["FaZe Clan", "NaVi", "Cloud9", "ENCE", "Astralis", "Vitality", "MR", "G2", "NiP", "MOUZ", "FURIA", "BIG", "Heroic", "Liquid", "Spirit"]
# Check if any are missing
missing_teams = [team for team in team_list if team not in dataframe["Team"].unique()]
if missing_teams:
    print("Missing teams:", missing_teams)
    raise ValueError("Missing teams")
dataframe = dataframe[dataframe["Team"].isin(team_list)]

## Download CFGS

In [33]:
import os
import requests
import threading

from bs4 import BeautifulSoup


# Extract data
def get_player_data(row):
    # Go to the player's profile page (https://prosettings.net/players/{player})
    player = row["Player"]
    team = row["Team"]
    player_url = f"https://prosettings.net/players/{player}"
    response = requests.get(player_url)
    soup = BeautifulSoup(response.text, "html.parser")

    settings_dict = {}

    # Each category is found as follows:
    # Section -> Table -> Data in rows

    # Mouse
    mouse_section = soup.find("section", id="csgo_mouse")
    mouse_table = mouse_section.find("table", class_="settings")
    for row in mouse_table.find_all("tr"):
        key = row["data-field"]
        value = row.find("td").text
        settings_dict[key] = value

    # Crosshair
    crosshair_section = soup.find("section", id="csgo_crosshair")
    crosshair_table = crosshair_section.find("table", class_="settings")
    for row in crosshair_table.find_all("tr"):
        key = row["data-field"]
        value = row.find("td").text
        settings_dict[key] = value

    # Viewmodel
    viewmodel_section = soup.find("section", id="csgo_viewmodel")
    viewmodel_table = viewmodel_section.find("table", class_="settings")
    for row in viewmodel_table.find_all("tr"):
        key = row["data-field"]
        value = row.find("td").text
        settings_dict[key] = value

    # Bob
    bob_section = soup.find("section", id="csgo_bob")
    bob_table = bob_section.find("table", class_="settings")
    for row in bob_table.find_all("tr"):
        key = row["data-field"]
        value = row.find("td").text
        settings_dict[key] = value

    # Video Settings
    video_section = soup.find("section", id="csgo_video_settings")
    video_table = video_section.find("table", class_="settings")
    for row in video_table.find_all("tr"):
        key = row["data-field"]
        value = row.find("td").text
        settings_dict[key] = value

    # Advanced Video
    advanced_video_section = soup.find("section", id="advanced_video")
    advanced_video_table = advanced_video_section.find("table", class_="settings")
    for row in advanced_video_table.find_all("tr"):
        key = row["data-field"]
        value = row.find("td").text
        settings_dict[key] = value

    # Create pro_configs/{team} if it doesn't exist
    if not os.path.exists(f"pro_configs/{team}"):
        os.makedirs(f"pro_configs/{team}")

    # Save to cfg file
    with open(f"pro_configs/{team}/{player}.cfg", "w") as f:
        f.write(f"// {player} CFG\n")
        for key, value in settings_dict.items():
            f.write(f"{key} {value}\n")

    with open(f"pro_configs/{team}/{player}.cfg", "a") as f:
        f.write("\n// Print settings\nclear\n")
        for key, value in settings_dict.items():
            f.write(f"echo {key}: {value}\n")

# Test with one player (electronic)
# get_player_data(dataframe.iloc[0])
# print(f"Done with {dataframe.iloc[0]['Player']}")

# Run with all players
players = dataframe
threads = []

for index, row in players.iterrows():
    thread = threading.Thread(target=get_player_data, args=(row,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

## Pick a random CFG

In [46]:
# Pick random .cfg from pro_configs/*
import random
import os

# Get list of all .cfg files
cfgs = []
for root, dirs, files in os.walk("pro_configs"):
    for file in files:
        if file.endswith(".cfg"):
            cfgs.append(os.path.join(root, file))

# Pick random .cfg
random_cfg = random.choice(cfgs)
print(random_cfg)

pro_configs\FaZe Clan\karrigan.cfg
